# Titanic simple model

In [54]:
from pandas.core.common import SettingWithCopyWarning
import warnings
from tensorflow import keras
import utils
import pandas as pd

warnings.filterwarnings("ignore", category=SettingWithCopyWarning)

In [55]:
from loadDataUtils import loadDataUtils

In [56]:
path_train = r'C:\Users\39320\Desktop\myProjects_python\Titanic\data\train.csv'
path_test = r'C:\Users\39320\Desktop\myProjects_python\Titanic\data\test.csv'
data = loadDataUtils(path_train, path_test)

In [57]:
df_train, df_test = data.get_train_and_test()

### Clean data

In [58]:
from titanicPreprocessing import preprocess
import numpy as np

In [59]:
p = preprocess(df_train.copy(), df_test.copy())
p.do_preprocess()

In [60]:
train, test = p.get_data()

In [61]:
train_target = train['Survived']
train.drop(columns=['Survived'], inplace=True)

In [62]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   PassengerId      889 non-null    int64  
 1   Pclass           889 non-null    int64  
 2   Age              889 non-null    float64
 3   SibSp            889 non-null    int64  
 4   Parch            889 non-null    int64  
 5   Fare             889 non-null    float64
 6   cabin_multiple   889 non-null    int64  
 7   Sex_female       889 non-null    uint8  
 8   Sex_male         889 non-null    uint8  
 9   Embarked_C       889 non-null    uint8  
 10  Embarked_Q       889 non-null    uint8  
 11  Embarked_S       889 non-null    uint8  
 12  cabin_letter_0   889 non-null    uint8  
 13  cabin_letter_A   889 non-null    uint8  
 14  cabin_letter_B   889 non-null    uint8  
 15  cabin_letter_C   889 non-null    uint8  
 16  cabin_letter_D   889 non-null    uint8  
 17  cabin_letter_E  

In [63]:
test.insert(20, 'cabin_letter_T', 0)

In [64]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   PassengerId      418 non-null    int64  
 1   Pclass           418 non-null    int64  
 2   Age              418 non-null    float64
 3   SibSp            418 non-null    int64  
 4   Parch            418 non-null    int64  
 5   Fare             418 non-null    float64
 6   cabin_multiple   418 non-null    int64  
 7   Sex_female       418 non-null    uint8  
 8   Sex_male         418 non-null    uint8  
 9   Embarked_C       418 non-null    uint8  
 10  Embarked_Q       418 non-null    uint8  
 11  Embarked_S       418 non-null    uint8  
 12  cabin_letter_0   418 non-null    uint8  
 13  cabin_letter_A   418 non-null    uint8  
 14  cabin_letter_B   418 non-null    uint8  
 15  cabin_letter_C   418 non-null    uint8  
 16  cabin_letter_D   418 non-null    uint8  
 17  cabin_letter_E  

## Drop not important columns

In [65]:
train.drop(columns=['Embarked_C'], inplace=True)
train.drop(columns=['Embarked_Q'], inplace=True)
train.drop(columns=['Embarked_S'], inplace=True)

train.drop(columns=['cabin_letter_0'], inplace=True)
train.drop(columns=['cabin_letter_A'], inplace=True)
train.drop(columns=['cabin_letter_B'], inplace=True)
train.drop(columns=['cabin_letter_C'], inplace=True)
train.drop(columns=['cabin_letter_D'], inplace=True)
train.drop(columns=['cabin_letter_E'], inplace=True)
train.drop(columns=['cabin_letter_F'], inplace=True)
train.drop(columns=['cabin_letter_G'], inplace=True)
train.drop(columns=['cabin_letter_T'], inplace=True)

test.drop(columns=['Embarked_C'], inplace=True)
test.drop(columns=['Embarked_Q'], inplace=True)
test.drop(columns=['Embarked_S'], inplace=True)

test.drop(columns=['cabin_letter_0'], inplace=True)
test.drop(columns=['cabin_letter_A'], inplace=True)
test.drop(columns=['cabin_letter_B'], inplace=True)
test.drop(columns=['cabin_letter_C'], inplace=True)
test.drop(columns=['cabin_letter_D'], inplace=True)
test.drop(columns=['cabin_letter_E'], inplace=True)
test.drop(columns=['cabin_letter_F'], inplace=True)
test.drop(columns=['cabin_letter_G'], inplace=True)
test.drop(columns=['cabin_letter_T'], inplace=True)

In [66]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train, train_target, test_size=0.3, random_state=42)

In [67]:
param_grid_svc = tuned_parameters = [{'kernel': ['rbf'], 'gamma': [.1,.5,1,2,5,10],
                                  'C': [.1, 1, 10, 100, 1000]},
                                 {'kernel': ['linear'], 'C': [.1, 1, 10, 100, 1000]},
                                 {'kernel': ['poly'], 'degree' : [2,3,4,5], 'C': [.1, 1, 10, 100, 1000]}]

param_grid_rf =  {'n_estimators': [100,500,1000],
                                  'bootstrap': [True,False],
                                  'max_depth': [3,5,10,20,50,75,100,None],
                                  'max_features': ['auto','sqrt'],
                                  'min_samples_leaf': [1,2,4,10],
                                  'min_samples_split': [2,5,10]}

param_grid_xgb = {
    'n_estimators': [20, 50, 100, 250, 500,1000],
    'colsample_bytree': [0.2, 0.5, 0.7, 0.8, 1],
    'max_depth': [2, 5, 10, 15, 20, 25, None],
    'reg_alpha': [0, 0.5, 1],
    'reg_lambda': [1, 1.5, 2],
    'subsample': [0.5,0.6,0.7, 0.8, 0.9],
    'learning_rate':[.01,0.1,0.2,0.3,0.5, 0.7, 0.9],
    'gamma':[0,.01,.1,1,10,100],
    'min_child_weight':[0,.01,0.1,1,10,100],
    'sampling_method': ['uniform', 'gradient_based']
}

In [68]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
from sklearn.metrics import accuracy_score
from utils import clf_performance

In [69]:
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [70]:
svc_g = SVC(probability = True)
rf_g = RandomForestClassifier(random_state = 1)
xgb_g = XGBClassifier(random_state = 1)

In [71]:
clf_svc = GridSearchCV(svc_g, param_grid = param_grid_svc, cv = 5, verbose = True, n_jobs = -1)
best_clf_svc = clf_svc.fit(X_train.loc[:, X_train.columns != 'PassengerId'],y_train)
clf_performance(best_clf_svc,'SVC')

Fitting 5 folds for each of 55 candidates, totalling 275 fits
SVC
Best Score: 0.8279870967741936
Best Parameters: {'C': 10, 'degree': 3, 'kernel': 'poly'}


In [72]:
svc_best = SVC(C= 10, degree=3, kernel= 'poly', probability=True)
svc_best.fit(X_train.loc[:, X_train.columns != 'PassengerId'],y_train)

y_pred_best = svc_best.predict(X_test.loc[:, X_test.columns != 'PassengerId'])
accuracy_score(y_pred_best, y_test)

0.8014981273408239

In [73]:
# very long to perform
clf_rf = RandomizedSearchCV(rf_g, param_distributions = param_grid_rf, cv = 5, verbose = True, n_jobs = -1)
best_clf_rf = clf_rf.fit(X_train.loc[:, X_train.columns != 'PassengerId'],y_train)
clf_performance(best_clf_rf,'Random Forest')

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Random Forest
Best Score: 0.8409032258064517
Best Parameters: {'n_estimators': 100, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 50, 'bootstrap': True}


In [74]:
from sklearn.ensemble import RandomForestClassifier

rf_best = RandomForestClassifier(bootstrap= True, max_depth= 50, max_features= 'auto', min_samples_leaf= 1, min_samples_split= 10, n_estimators= 100)
rf_best.fit(X_train.loc[:, X_train.columns != 'PassengerId'],y_train)
pred_rf_best = rf_best.predict(X_test.loc[:, X_test.columns != 'PassengerId'])
accuracy_score(pred_rf_best, y_test)

0.8014981273408239

In [75]:
clf_xgb = RandomizedSearchCV(xgb_g, param_distributions = param_grid_xgb, cv = 5, verbose = True, n_jobs = -1)
best_clf_xgb = clf_xgb.fit(X_train.loc[:, X_train.columns != 'PassengerId'],y_train)
clf_performance(best_clf_xgb,'XGB')

Fitting 5 folds for each of 10 candidates, totalling 50 fits
XGB
Best Score: 0.8472903225806452
Best Parameters: {'subsample': 0.9, 'sampling_method': 'uniform', 'reg_lambda': 2, 'reg_alpha': 0.5, 'n_estimators': 20, 'min_child_weight': 1, 'max_depth': 10, 'learning_rate': 0.3, 'gamma': 1, 'colsample_bytree': 1}


C:\Users\39320\Desktop\myProjects_python\Titanic\venvimg\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
20 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\39320\Desktop\myProjects_python\Titanic\venvimg\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\39320\Desktop\myProjects_python\Titanic\venvimg\lib\site-packages\xgboost\core.py", line 532, in inner_f
    return f(**kwargs)
  File "C:\Users\39320\Desktop\myProjects_python\Titanic\venvimg\lib\site-packages\xgboost\sklearn.py

In [76]:
xgb_best = XGBClassifier(colsample_bytre= 0.8, gamma= 0.01, learning_rate= 0.1, max_depth= 5, min_child_weight= 1, n_estimators= 250, reg_alpha= 1, reg_lambda= 1, sampling_method= 'uniform', subsample= 0.5)
xgb_best.fit(X_train.loc[:, X_train.columns != 'PassengerId'],y_train)

y_pred_best_xgb = xgb_best.predict(X_test.loc[:, X_test.columns != 'PassengerId'])
accuracy_score(y_pred_best_xgb, y_test)

[16:09:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_bytre" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




0.8089887640449438

In [77]:
voting_clf_best = VotingClassifier(estimators = [('rf',rf_best),('svc',svc_best), ('xgb', xgb_best)], voting = 'soft', weights=[2,2,3])
voting_clf_best.fit(X_train.loc[:, X_train.columns != 'PassengerId'],y_train)
p_v_best = voting_clf_best.predict(X_test.loc[:, X_test.columns != 'PassengerId'])
accuracy_score(p_v_best, y_test)

[16:09:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_bytre" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




0.8127340823970037

In [78]:
svc_best.fit(train.loc[:, train.columns != 'PassengerId'], train_target)
xgb_best.fit(train.loc[:, train.columns != 'PassengerId'], train_target)
voting_clf_best.fit(train.loc[:, train.columns != 'PassengerId'], train_target)

[16:09:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_bytre" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[16:09:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_bytre" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(max_depth=50,
                                                     min_samples_split=10)),
                             ('svc',
                              SVC(C=10, kernel='poly', probability=True)),
                             ('xgb',
                              XGBClassifier(base_score=0.5, booster='gbtree',
                                            callbacks=None, colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytre=0.8,
                                            colsample_bytree=1,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metr...
                                            gpu_id=-1, grow_policy='depthwise',
                                        

In [79]:
import numpy as np

y_hat_svc_best = svc_best.predict(test.loc[:, test.columns != 'PassengerId']).astype(int)
y_hat_xgb_best = xgb_best.predict(test.loc[:, test.columns != 'PassengerId']).astype(int)
y_hat_voting_clf_best = voting_clf_best.predict(test.loc[:, test.columns != 'PassengerId']).astype(int)

In [80]:
final_data = {'PassengerId': test.PassengerId, 'Survived': y_hat_svc_best}
submission = pd.DataFrame(data=final_data)

final_data_2 = {'PassengerId': test.PassengerId, 'Survived': y_hat_xgb_best}
submission_2 = pd.DataFrame(data=final_data_2)

final_data_3 = {'PassengerId': test.PassengerId, 'Survived': y_hat_voting_clf_best}
submission_3 = pd.DataFrame(data=final_data_3)

In [81]:
#prepare submission files
submission.to_csv('./submission_simple/submission_svc.csv', index =False)
submission_2.to_csv('./submission_simple/submission_xgb.csv',index=False)
submission_3.to_csv('./submission_simple/submission_voting.csv', index=False)